In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import bcolz
import pickle
import re
import math
import pandas as pd
from tqdm import tqdm
import pytorch_lightning as pl
#Dataset
from torch.utils.data import Dataset
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import sys
import os
# from gensim.corpora import WikiCorpus
import glob
# import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

torch.manual_seed(1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# # import pre-trained glove
# words = []
# idx = 0
# word2idx = {}

# vectors = bcolz.carray(np.zeros(1), rootdir=f'D:\Jupyter\Glove6B/6B.{dimension}.dat', mode = "w")

# with open(f'D:\Jupyter\Glove6B/glove.6B.{dimension}d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(float)
#         vectors.append(vect)

# vectors = bcolz.carray(vectors[1:].reshape((-1, 50)), rootdir=f'D:\Jupyter\Glove6B/6B.{dimension}.dat', mode = "w")
# vectors.flush()

# pickle.dump(words, open(f'D:\Jupyter\Glove6B/6B.{dimension}_words.pk1', 'wb'))
# pickle.dump(word2idx, open(f'D:\Jupyter\Glove6B/6B.{dimension}_idx.pk1', 'wb'))

# vectors = bcolz.open(f'D:\Jupyter\Glove6B/6B.{dimension}.dat')[:]
# words = pickle.load(open(f'D:\Jupyter\Glove6B/6B.{dimension}_words.pk1', 'rb'))
# word2idx = pickle.load(open(f'D:\Jupyter\Glove6B/6B.{dimension}_idx.pk1', 'rb'))

# glove = {w: vectors[word2idx[w]] for w in words}

In [8]:
# train_sentence = """When forty winters shall besiege thy brow,
# And dig deep trenches in thy beauty's field,
# Thy youth's proud livery so gazed on now,
# Will be a totter'd weed of small worth held:
# Then being asked, where all thy beauty lies,
# Where all the treasure of thy lusty days;
# To say, within thine own deep sunken eyes,
# Were an all-eating shame, and thriftless praise.
# How much more praise deserv'd thy beauty's use,
# If thou couldst answer 'This fair child of mine
# Shall sum my count, and make my old excuse,'
# Proving his beauty by succession thine!
# This were to be new made when thou art old,
# And see thy blood warm when thou feel'st it cold.""".split()

# EMBEDDING_DIM = dimension

# vocab = set(train_sentence)
# vocab_size = len(vocab)
# weights_matrix = np.zeros((vocab_size, dimension))
# words_found = 0

# for i, word in enumerate(vocab):
#     try: 
#         weights_matrix[i] = glove[word]
#         words_found += 1
#     except KeyError:
#         weights_matrix[i] = np.random.normal(scale=0.6, size = (EMBEDDING_DIM, ))

In [10]:
# def create_emb_layer(weights_matrix, non_trainable = False):
#     num_embeddings, embedding_dim = weights_matrix.size()
#     emb_layer = nn.Embedding(num_embeddings, embedding_dim)
#     emb_layer.load_state_dict({'weight': weights_matrix})
#     if non_trainable:
#         emb_layer.weight.requires_grad = False
    
#     return emb_layer, num_embeddings, embedding_dim

# class ToyNN(nn.Module):
#     def __init__(self, weights_matrix, hidden_size, num_layers):
#         super(self).__init__()
#         self.embedding, num_embeddings, embedding_dim = create_emb_labyer(weights_matrix, True)
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first = True)
        
#     def forward(self, inp, hidden):
#         return self.gru(self.embedding(inp), hidden)
    
#     def init_hidden(self, batch_size):
#         return Variable(torch.zeros(self.num_layers, batch_size, self.heiidn_size))

In [2]:
def fnv1a(txt):
    # 64 bit fnv-1a
    txt = bytes(txt, 'utf-8')
    hval = 0xcbf29ce484222325
    fnv_prime = 0x100000001b3
    for c in txt:
        hval = hval ^ c
        hval = (hval * fnv_prime) % K
    return hval        

def subword_hashes(word):
    sub_hash = []
    tword = '<' + word + '>'
    sub_hash.append(fnv1a(tword))
    for n in range(3,7):
        for i in range(len(tword)-n+1):
            sub_hash.append(fnv1a(tword[i:i+n]))
    return sub_hash
            

In [6]:
class fasttext_dataset(Dataset):
    def __init__(self, in_f, SOURCE, CONTEXT_SIZE, SUBSAMPLING, REJ_THRESHOLD, MIN_FREQ, FORMAT, limit = None):
        vocab, text_dict = self.make_dict(in_f, SOURCE, SUBSAMPLING, REJ_THRESHOLD, MIN_FREQ, FORMAT, limit)
        training_data = self.get_training_data(text_dict, CONTEXT_SIZE)
        
        self.vocab = vocab
        self.text_dict = text_dict
        
        self.data = torch.tensor(training_data, dtype = torch.long)
    
    def __getitem__(self, idx):
        target = self.data[idx, 0]
        context = self.data[idx, 1]
        return target, context
    
    def __len__(self):
        return len(self.data)
    
    def make_dict(self, in_f, source, subsampling, rej_threshold, min_freq, FORMAT, limit):
        print("Gathering texts....")
        body_tag = FORMAT[source]
        t_dict = {}
        text_list = [None]
        for path in tqdm(glob.glob(in_f)):
            if limit is not None:
                limit -= 1
                if limit < 0:
                    break
            f = open(path, errors = 'ignore')
            data = f.read()
            file_type = path[len(path) -3:]
            if file_type == 'xml':
                root = BeautifulSoup(data, 'xml')
                paragraphs = root.find_all(body_tag)
            elif file_type == 'txt':
                paragraphs = [data]
            else:
                paragraphs = [data]
                
            for x in paragraphs:
                text = x.text.lower()
                words = word_tokenize(text)
                for w in words:
                    text_list.append(w)
                text_list.append(None)
            f.close()
            
#         print("Lemmatizing and removing stopwords...")
#         lem_text_list = []
#         lem = WordNetLemmatizer()
#         stop_words = set(stopwords.words('english'))
#         stop_words = set()
#         for w in tqdm(text_list):
#             if w is None:
#                 lem_text_list.append(w)
#                 continue
#             w = lem.lemmatize(w, 'v')
#             if w not in stop_words:
#             lem_text_list.append(w)
        voc, text_list = self.get_word_freqs(text_list, subsampling, rej_threshold, min_freq)
#         print("\n\n\n", vocab)
        print("Constructing Dictionary...")
        for idx, w in tqdm(enumerate(text_list)):
            t_dict[idx] = w
#         print(text_dict)
        print("Dictionary creation completed.")
        return voc, t_dict
    
    def get_training_data(self, text_dict, context_size):
        training_data = []
        for t_idx in range(len(text_dict)):
            if text_dict[t_idx] is None:
                continue
            for sign in [-1,1]:
                for w in range(1, context_size+1):
                    c_idx = t_idx + w*sign
                    if text_dict[c_idx] is None:
                        break
                    training_data.append([t_idx, c_idx])
        return training_data
    
    def get_word_freqs(self, text_list, subsampling, rej_threshold, min_freq):
        v = {}
        total = 0.0
        new_text_list = []
        for word in text_list:
            if word is not None:
                if word not in v:
                    v[word] = 0.0
                v[word] += 1.0
                total += 1.0
#         print("\nSubsampling: ", subsampling)
# #         a = 100
# #         print(len(text_list))
#         if subsampling:
#             print("Subsampling...")
#             for w in tqdm(text_list):
# #                 print(word)
#                 word = w
#                 if word is not None:
#                     fq = v[word]/total
#                     prob = 1 - np.sqrt(rej_threshold/fq)
#                     sampling = np.random.sample()
#                     print(word, prob, sampling)
#                     if sampling < prob or v[word] < min_freq:
#                         text_list.remove(word)
#             print(len(text_list))
        return v, text_list

In [7]:
in_folder = "D:/VISTEC Intern/corpus/blogs/*"
SOURCE = 'blog'
CONTEXT_SIZE = 3
SUBSAMPLING = True
REJ_THRESHOLD = 1e-5
MIN_FREQ = 2
FORMAT = {'blog':'post'}

# total embeddings
K = int(2e6)

dimension = 50
neg_samples = 5

In [8]:
train_dataset = fasttext_dataset(in_folder, SOURCE, CONTEXT_SIZE, SUBSAMPLING, REJ_THRESHOLD, 
                                 MIN_FREQ, FORMAT, limit = 10)
# print(train_dataset.text_dict)

  0%|                                                                                             | 0/19320 [00:00<?, ?it/s]

Gathering texts....


  0%|                                                                                    | 10/19320 [00:01<56:09,  5.73it/s]
229307it [00:00, 2669810.65it/s]


Constructing Dictionary...
Dictionary creation completed.


In [104]:
class FastText_Model(pl.LightningModule):
    
    def __init__(self, subword_size, embedding_dim, noise_dist, neg_samples, text_dict, vocab, debug = False):
        super().__init__()
        self.subword_embedding = nn.Embedding(subword_size, embedding_dim)
        self.neg_samples = neg_samples
        self.dimension = embedding_dim
        self.embedding_size = subword_size
        self.noise_dist = noise_dist
        self.text_dict = text_dict
        self.vocab = vocab
        self.debug = debug
        self.subword_embedding.weight.data.uniform_(-0.05,0.05)
    
    def forward(self, target_idx, context_idx):
        debug = self.debug
#         print(target_idx.shape, context_idx.shape)
        first = True
        for idx in target_idx.tolist():
            target = self.text_dict[idx]
#             print(target)
            t_hash = torch.tensor(subword_hashes(target), dtype = torch.long)
#             print(t_hash)
#             print('t_hash.shape: ', t_hash.shape)
#             print(self.subword_embedding(t_hash))
            t_vec = torch.sum(self.subword_embedding(t_hash), dim = 0)
#             print('t_vec.shape: ', t_vec.shape)
            if first:
                batch_t_vec = t_vec.detach().clone()
                first = False
            else:
                batch_t_vec = torch.cat((batch_t_vec,t_vec), dim = 0)
#         print(self.dimension)
        batch_t_vec = batch_t_vec.view(target_idx.shape[0], self.dimension)
        if debug:print("batch_t_vec.shape: ", batch_t_vec.shape)
        
        first = True
        for idx in context_idx.tolist():
            context = self.text_dict[idx]
            c_hash = torch.tensor(subword_hashes(context), dtype = torch.long)
#             print('c_hash.shape: ', c_hash.shape)
            c_vec = torch.sum(self.subword_embedding(c_hash), dim = 0)
#             print('c_vec.shape: ', c_vec.shape)
            if first:
                batch_c_vec = c_vec.detach().clone()
                first = False
            else:
                batch_c_vec = torch.cat((batch_c_vec,c_vec), dim = 0)
        batch_c_vec = batch_c_vec.view(context_idx.shape[0], self.dimension)
        if debug:print("batch_c_vec.shape: ", batch_c_vec.shape)
        
        # negatives    
# excluding context words from negative samples
#         cts = set()
#         for sign in [-1,1]:
#             for w in range(1, CONTEXT_SIZE+1):
#                 c_idx = target_idx + w*sign
#                 if text_dict[c_idx] is None:
#                     break
#                 cts.add(text_dict[c_idx])
#         W = {}
#         P_vec = []
#         for w in self.noise_dist:
#             if w not in cts:
#                 total += self.noise_dist[w]
                
#         for i,w in enumerate(self.noise_dist):
#             if w not in cts:
#                 fq = self.noise_dist[w]/total
#                 prob = 1 - np.sqrt(REJ_THRESHOLD/fq)
#                 W[i] = w
#                 P_vec.append(prob)
#         distr = torch.tensor(P_vec, dtype = torch.long)

        distr = self.noise_dist
        ngs = torch.multinomial(distr, context_idx.shape[0]*self.neg_samples, replacement = True)
        if debug: print('negative_samples.shape', ngs.shape)
            
        ngs = ngs.view(context_idx.shape[0], self.neg_samples)
        if debug: print('negative_samples.shape', ngs.shape)
            
#         print(noise_dist)
        keys = list(self.vocab.keys())
        first_neg = True
        for neg_c in ngs.tolist():
#             if debug: print("neg_c", neg_c)
            ns_words = [keys[i] for i in neg_c]
            first = True
            for word in ns_words:
                nc_hash = torch.tensor(subword_hashes(word), dtype = torch.long)
#                 if debug:print('n_c_hash.shape: ', nc_hash.shape)
                nc_vec = torch.sum(self.subword_embedding(nc_hash), dim = 0)
#                 if debug:print('n_c_vec.shape: ', nc_vec.shape)
                if first:
                    batch_nc_vec = nc_vec.detach().clone()
                    first = False
                else:
                    batch_nc_vec = torch.cat((batch_nc_vec,nc_vec), dim = 0)
            if first_neg:
                batch_n_vec = batch_nc_vec.detach().clone()
                first_neg = False
            else:
                batch_n_vec = torch.cat((batch_n_vec,batch_nc_vec), dim = 0)
            
#             print("batch_nc_vec.shape: ", batch_nc_vec.shape)
        batch_n_vec = batch_n_vec.view(target_idx.shape[0], self.neg_samples, self.dimension)
        if debug: print("batch_n_vec.shape: ", batch_n_vec.shape)
        return batch_t_vec, batch_c_vec, batch_n_vec
    
    def training_step(self, batch, batch_idx):
        print(f"\n###### BATCH {batch_idx} ######")
        debug = self.debug
        
        x, y = batch
        if debug:
            print("x.shape: ", x.shape)
            print("y.shape: ", y.shape)
        t_vec, c_vec, n_vec = self(x,y)
        if debug:print("t_vec, c_vec, n_vec.shape\n", t_vec.shape, c_vec.shape, n_vec.shape)
        
        emb_product = torch.mul(t_vec, c_vec)
        if debug:print("positive product.shape", emb_product.shape)
            
        emb_product = torch.sum(emb_product, dim=1)
        if debug:print("positive sum.shape", emb_product.shape)
            
        pos_loss = F.logsigmoid(emb_product)
        if debug:print("pos_loss.shape", pos_loss.shape)
            
        # negative
        if debug:print("\nt_vec.unsqueeze(2).shape: ", t_vec.unsqueeze(2).shape)
        neg_product = torch.bmm(n_vec.neg(), t_vec.unsqueeze(2))
        if debug:print("nega_product.shape", neg_product.shape)

        neg_loss = F.logsigmoid(neg_product).squeeze(2).sum(1)
        if debug:print("neg_loss.shape", neg_shape.shape)

        total_loss = -(pos_loss + neg_loss).mean()
        if debug:print("total_loss.shape", total_loss.shape)
           
        print("total_loss", total_loss)
        
        return total_loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 0.01)
    

In [105]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 20)

word_freqs = np.array(list(train_dataset.vocab.values()))
unigram_dist = word_freqs/sum(word_freqs)
noise_dist = torch.from_numpy(unigram_dist**(0.75)/np.sum(unigram_dist**(0.75)))

trainer = pl.Trainer()
model = FastText_Model(K, dimension, noise_dist, neg_samples, train_dataset.text_dict, train_dataset.vocab, False)
trainer.fit(model, train_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type      | Params
------------------------------------------------
0 | subword_embedding | Embedding | 100 M 
------------------------------------------------
100 M     Trainable params
0         Non-trainable params
100 M     Total params
400.000   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]


###### BATCH 0 ######
total_loss tensor(4.1730, grad_fn=<NegBackward>)

###### BATCH 1 ######
total_loss tensor(4.1737, grad_fn=<NegBackward>)

###### BATCH 2 ######
total_loss tensor(4.0967, grad_fn=<NegBackward>)

###### BATCH 3 ######
total_loss tensor(4.0943, grad_fn=<NegBackward>)

###### BATCH 4 ######
total_loss tensor(4.0876, grad_fn=<NegBackward>)

###### BATCH 5 ######
total_loss tensor(4.1068, grad_fn=<NegBackward>)

###### BATCH 6 ######
total_loss tensor(4.2923, grad_fn=<NegBackward>)

###### BATCH 7 ######
total_loss tensor(4.1979, grad_fn=<NegBackward>)

###### BATCH 8 ######
total_loss tensor(4.4161, grad_fn=<NegBackward>)

###### BATCH 9 ######
total_loss tensor(4.7129, grad_fn=<NegBackward>)

###### BATCH 10 ######
total_loss tensor(4.5952, grad_fn=<NegBackward>)

###### BATCH 11 ######
total_loss tensor(4.4375, grad_fn=<NegBackward>)

###### BATCH 12 ######
total_loss tensor(4.4816, grad_fn=<NegBackward>)

###### BATCH 13 ######
total_loss tensor(4.5731, grad_fn=<Ne

In [ ]:
fq = np.array([1,2,3])
ud = fq/sum(fq)
nd = torch.from_numpy(ud/np.sum(ud))
xd = np.array([1]*5)
print(np.sqrt(1/xd))
print(1-xd)
print(np.array([1,2,3]) - np.array([1,1,1]))

In [ ]:
f = torch.tensor([[1,2,3],[4,5,6]], dtype = torch.long)
g = torch.tensor([[7,8,9],[10,11,12]], dtype = torch.long)
print(torch.sum(f,dim=0))
h = torch.cat((f,g))
print(h.shape)
h = h.view(2,2,3)
print(h.shape)
print(h.unsqueeze(2))
    
# e = f.detach().clone()
# e = torch.cat((e,f),dim=0)
# print(e)
# print(f)

# print(torch.cat((e,f), dim = 0))
# print(f.shape)
# print(torch.sum(f, dim=0))
# print(torch.cat((f,f), dim =0).shape)

In [31]:
class test(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(20, 5)
        self.embed.weight.data.uniform_(-0.05,0.05)
        
    def forward(self):
        idx = torch.tensor([0,1], dtype = torch.long)
        multiplier = self.embed(idx)
        target = torch.tensor([[i for i in range(5)]], dtype = torch.long)
        print("idx.shape: ", idx.shape)
        print("idx: ", idx)
        print("\nmultiplier.shape: ", multiplier.shape)
        print("multiplier: ", multiplier)
        print("\ntarget.shape: ", target.shape)
        print("target: ", target)
        
        product = torch.mul(target, multiplier)
        print("\nproduct.shape: ", product.shape)
        print("product: ", product)
        emb_sum = torch.sum(product, dim=1)
        print("\nsum.shape: ", emb_sum.shape)
        print("sum: ", emb_sum)
        
        noise_dist = torch.ones(20)
        ng = torch.multinomial(noise_dist,5, replacement = True)
        
        return

In [ ]:
tt = test()
# optimizer = optim.Adam(tt.parameteres(), lr = 0.001)
tt.train()
# optimizer.zerp_grad()
loss = tt()

In [13]:
a = {'b':2,'a':1}
print(list(a.keys())[1])

a


In [7]:
x = [1,2,3]
print(torch.tensor([x], dtype = torch.long).shape)
print(torch.tensor([i for i in x], dtype = torch.long).shape)

torch.Size([1, 3])
torch.Size([3])


In [96]:
a = {'a':1, 'b':2, 'c':20}
b = [word for word in a]
print(b)

['a', 'b', 'c']


In [14]:
a = {1,2,3}
a.add(1)
print(a)

{1, 2, 3}


In [ ]:
a = torch.tensor([[1,0,0,0],[2,0,0,0],[3,0,0,0],[4,0,0,0]])
v = torch.tensor([[0,0,0,1],[0,0,0,2],[0,0,0,3],[0,0,0,4]])
xc = torch.tensor([[1,2,1,1]])
sv = torch.tensor([[1,0,0,0],[1,2,0,0]])